In [2]:
from importlib import reload
import Tradables
reload(Tradables)
from Tradables import Underlier
import pickle
import gs_quant
from gs_quant.session import GsSession
from datetime import datetime, timedelta
import investpy
from gs_quant.analytics.datagrid import DataColumn
from gs_quant.analytics.processors import LastProcessor, EntityProcessor
from gs_quant.data.coordinate import DataCoordinate, DataMeasure, DataFrequency
from gs_quant.analytics.datagrid import DataRow
from gs_quant.markets.securities import Asset, AssetIdentifier
from gs_quant.analytics.datagrid import DataColumn, DataRow, DimensionsOverride, DataGrid
from gs_quant.analytics.processors import LastProcessor, EntityProcessor, CoordinateProcessor
from gs_quant.data import DataDimension
from gs_quant.data.coordinate import DataCoordinate, DataMeasure, DataFrequency
from gs_quant.markets.securities import AssetIdentifier, Asset
from gs_quant.session import GsSession
import pandas as pd
import os
import yfinance as yf
from datetime import datetime, timedelta

"""import EURUSD data"""
def save_security(sec):
    """save security to pickle in the securities folder"""
    with open(f"Securities/{sec.name}.pkl", "wb") as f:
        pickle.dump(sec, f)
GsSession.use(
    client_id="3e871897e5474a0488cb09ea699ffe1c",
    client_secret="3dd2854652249d480cb7e8724ff23ee892dde030da367d24917565183b2ae929"
)
date_offsets = {
    '1D': timedelta(days=1),
    '1W': timedelta(weeks=1),
    '2W': timedelta(weeks=2),
    '3W': timedelta(weeks=3),
    '1M': timedelta(days=30),
    '2M': timedelta(days=60),
    '3M': timedelta(days=90),
    '6M': timedelta(days=180),
    '9M': timedelta(days=270),
    '1Y': timedelta(days=365),
    'Overnight': timedelta(days=1),
    'Tomorrow Next': timedelta(days=1),
    'Spot Next': timedelta(days=2),
    'One Week': timedelta(weeks=1),
    'Two Weeks': timedelta(weeks=2),
    'Three Weeks': timedelta(weeks=3),
    'One Month': timedelta(days=30),
    'Two Months': timedelta(days=60),
    'Three Months': timedelta(days=90),
    'Four Months': timedelta(days=120),
    'Five Months': timedelta(days=150),
    'Six Months': timedelta(days=180),
    'Seven Months': timedelta(days=210),
    'Eight Months': timedelta(days=240),
    'Nine Months': timedelta(days=270),
    'Ten Months': timedelta(days=300),
    'Eleven Months': timedelta(days=330),
    'One Year': timedelta(days=365),
    'Two Years': timedelta(days=730),
    'Three Years': timedelta(days=1095),
    'Four Years': timedelta(days=1460),
    'Five Years': timedelta(days=1825),
    'Six Years': timedelta(days=2190),
    'Seven Years': timedelta(days=2555),
    'Ten Years': timedelta(days=3650),
}


"""use excel wings to load surface from EURUSD Risk&Marks"""
guipath = os.path.abspath('C:\\Users\\jacob\\bolt-hub\\GUI.xlsx')
marks = pd.read_excel(guipath, sheet_name="EURUSD Risks&Marks")



def save_security(sec):
    """save security to pickle in the securities folder"""
    with open(f"Securities/{sec.name}.pkl", "wb") as f:
        pickle.dump(sec, f)

def get_intraday_weighting():
    ticker_symbol = 'EURUSD=X'
    start_date = '2022-01-01'
    end_date = '2023-06-01'
    # Get the data from Yahoo Finance
    data = yf.download(ticker_symbol, start=start_date, end=end_date, interval="1h")

    # Extract the open and close prices by hour
    hourly_data = data[['Open', 'Close']].resample('H').last()

    """create a column representing hour of the day"""
    hourly_data['Hour'] = hourly_data.index.hour
    """create a column representing change in price from previous hour"""
    hourly_data['Change'] = abs(hourly_data['Open']-hourly_data['Close'])/hourly_data['Open']

    """group by hour and calculate mean change"""
    hourly_data = hourly_data.groupby('Hour')['Change'].mean()
    return hourly_data

def extract_forward_curve(marks):
    """extract forward curve"""
    fwd = marks.drop(index=range(5))
    last_row_index = fwd.index[-31]
    # Select all rows up to and including the last 31st row, and assign it back to the dataframe
    fwd = fwd.iloc[:last_row_index+1, :]
    fwd = fwd.iloc[:, :4]
    fwd.iloc[1:, 0] = [date_offsets[tenor] for tenor in fwd.iloc[1:, 0]]
    # get the values of the top row as a series
    new_columns = fwd.iloc[0]
    # assign the new column names to the dataframe
    fwd.columns = new_columns
    # drop the first row since it's no longer needed as column names
    fwd = fwd.iloc[1:]
    # set the index to the values in the first column by position
    fwd = fwd.set_index(fwd.columns[0])
    # drop the first column since it's now the index
    fwd = fwd.drop(columns=[fwd.columns[0]])
    fwd = fwd.rename_axis(index=None).rename_axis(columns=None)
    fwd.drop('Bid', axis=1, inplace=True)
    return fwd

def extract_vol_surface(marks):
    """extract vol surface"""
    vols = marks.drop(index=range(33))
    vols = vols.iloc[0:, :18]
    # get the values of the top row as a series
    new_columns = [5,10,15,20,25,30,35,40,45,50,-40,-35,-30,-25,-20,-15,-10,-5]
    # assign the new column names to the dataframe
    vols.columns = new_columns
    # drop the first row since it's no longer needed as column names
    vols = vols.iloc[1:]
    # set the index to the values in the first column by position
    vols = vols.set_index(vols.columns[0])
    # drop the first column since it's now the index
    vols = vols.drop(columns=[vols.columns[0]])
    vols = vols.rename_axis(index=None).rename_axis(columns=None)
    new_index = [date_offsets[tenor] for tenor in vols.index]
    vols.set_index(pd.Index(new_index), inplace=True)
    return vols


ModuleNotFoundError: No module named 'Tradables'

In [3]:
eurusd = Underlier("EURUSD")
eurusd.load_forward_curve(extract_forward_curve(marks))
eurusd.load_vol_surface(extract_vol_surface(marks))
eurusd.mark_spot(1.0935)
eurusd.load_intraday_weights(get_intraday_weighting())

save_security(eurusd)


NameError: name 'Underlier' is not defined

In [7]:
"""load eurusd from pickle"""
with open("Securities/EURUSD.pkl", "rb") as f:
    eurusd = pickle.load(f)

eurusd.forward_curve_as_fn()(1)

array(1.09933)